# ML2 Übung 4 - Deep Q-Netzwerke

In dieser Übung werden Deep Q-Netzwerke dazu verwendet einen Agenten zu trainieren. Als Umgebung für den Agenten wird die [PyGame Learning Environment (PLE)](https://github.com/ntasfi/PyGame-Learning-Environment) verwendet. Zuerst werden die notwendigen Imports durchgeführt.

In [ ]:
import tensorflow as tf
import random
import numpy as np
from ple import PLE
from dqn.environments.ple.ple_environment import VisualPleEnvironment
from dqn.agents.replay_memory import ReplayMemory
from dqn.agents.history import History
from config import TensorFlowConfig

Anschließend wird eine Konfiguration für die PLE-Umgebung instanziert.
Die untenstehenden Werte können angepasst werden, um die PLE-Umgebung anzupassen.
Die PLE-Umgebung wird gestartet und es sollte sich ein "pygame window" öffnen.

In [ ]:
config = TensorFlowConfig

config.env_name = "Pong" # eine Auswahl von gültigen Namen findet sich untenstehend
config.display = True # Kann auf False gesetzt werden, dadurch wird das Spiel nicht "gerendert", schneller im Training
config.action_repeat = 4
config.history_length = 4
config.state_width = 84
config.state_height = 84
config.cnn_format = "NHWC"

env = VisualPleEnvironment(config)
print env.get_game_names()

Nun wird die interaktive Session gestartet und der Tensorflow Graph mit der CNN Architektur erstellt.

In [ ]:
tf.reset_default_graph()
sess = tf.InteractiveSession()

In [ ]:
# Define function to create convolutional layers
def conv2d(x, output_dim, kernel_size, stride, initializer, activation_fn, name="conv2d"):
    stride = [1, stride[0], stride[1], 1]
    kernel_shape = [kernel_size[0], kernel_size[1], x.get_shape()[-1], output_dim]
    data_format = "NHWC"
    padding = "VALID"
    
    with tf.variable_scope(name):
        w = tf.get_variable("w", kernel_shape, tf.float32, initializer=initializer)
        # AUFGABE:
        # Erstelle eine 2D-Convolution mit x als Eingabe, den Gewichten W und dem obigen stride.
        # Setze zu dem das padding und das data_format!
        conv = None
        b = tf.get_variable("biases", [output_dim], initializer=tf.constant_initializer(0.0))
        
        # AUFGABE:
        # Definiere eine Funktion, um die Biases auf die Convolution zu addieren.
        out = None
    
    if activation_fn != None:
        out = activation_fn(out)
    return out, w, b

# Define function to create linear layers
def linear(x, output_dim, stddev=0.02, bias_start=0.0, activation_fn=None, name="linear"):
    shape = x.get_shape().as_list()
    
    with tf.variable_scope(name):
        w = tf.get_variable("w", [shape[1], output_dim], tf.float32, tf.random_normal_initializer(stddev=stddev))
        # AUFGABE:
        # Erstelle eine Matrizenmultiplikation mit x und den Gewichten W
        lin = None
        b = tf.get_variable("bias", [output_dim], initializer=tf.constant_initializer(bias_start))
        # AUFGABE:
        # Definiere eine Funktion, um die Biases auf die Matritzenmultiplikation zu addieren.
        out = None
        
    if activation_fn != None:
        out = activation_fn(out)
    return out, w, b

Nun wird das Netz zusammengesetzt:

In [ ]:
# Define default initializer and activation function
initializer = tf.truncated_normal_initializer(0, 0.02)
activation_fn = tf.nn.relu

In [ ]:
# Define training network
prediction_input = None
q = None
q_action = None
W = {}
with tf.variable_scope("prediction"):
    prediction_input = tf.placeholder("float32", 
                                      [None, config.state_height, config.state_width, config.history_length],
                                      name="prediction_input")
    l1, W["l1_w"], W["l1_b"] = conv2d(prediction_input, 32, [8,8], [4,4], initializer, activation_fn, name="l1")
    l2, W["l2_w"], W["l2_b"] = conv2d(l1, 64, [4,4], [2,2], initializer, activation_fn, name="l2")
    
    # AUFGABE:
    # Erstelle eine Layer l3 mit einem 3x3 Kernel, einem 1x1 Stride und 64 Ausgabefiltern.
    # Ansonsten analog zu l1 und l2
    l3, W["l3_w"], W["l3_b"] = None
    
    # flatten l3
    shape = l3.get_shape().as_list()
    l3_flat = tf.reshape(l3, [-1, reduce(lambda x, y: x * y, shape[1:])])
    
    l4, W["l4_w"], W["l4_b"] = linear(l3_flat, 512, activation_fn=activation_fn, name="l4")
    q, W["q_w"], W["q_b"] = linear(l4, env.action_size, name="q") # estimated q-values for the possible actions
    
    # action with the highest q-value
    # AUFGABE: Wähle den Index mit dem maximalen Q-Wert aus der Layer Q
    q_action = None 

Zusätzlich zu dem *Prediction*-Netzwerk wird in der Regel auch ein *Sollwert*- oder *Target*-Netzwerk verwendet.
Dies hat die gleiche Struktur das *Prediction*-Netzwerk, aber einen eigenen Satz von Gewichten.

In [ ]:
# Define target network
target_input = None
target_q = None
target_W = {}
with tf.variable_scope("target"):
    # AUFGABE:
    # Definiere das Target-Netzwerk.
    # Es hat genau die gleichen Schichten, wie das Prediction-Netzwerk.
    # Nur die Berechnung der q_action ist nicht notwendig.
    # Folgende Variablennamen verwenden: target_input statt prediction_input, target_q statt q und target_W statt W.
    # Achtung: Es ist wichtig, dass die Key-Namen der jeweiligen Gewichts-Dictionaries 
    # des Prediction- und Target-Netzwerks identisch sind.

Als nächstes wird eine Operation definiert, die die Gewichte aus einem gegeben Dictionary in die Gewichte des Target-Netzwerks überträgt.

In [ ]:
target_w_input = {}
target_w_assign_op = {}
with tf.variable_scope("pred_to_target"):
    for name in W.keys():
        target_w_input[name] = tf.placeholder("float32", target_W[name].get_shape().as_list(), name=name)
        # AUFGABE:
        # Füge hier den entsprechenden Code ein, der dem Eintrag aus target_W den Wert aus target_w_input "assigned".
        target_w_assign_op[name] = None

Als nächster Schritt wird eine Fehlerfunktion definiert. Hier wird der Huber-Loss verwendet. Ein anderer Loss würde aber auch funktionieren.

In [ ]:
def error(x):
    # Huber Loss
    try:
        return tf.select(tf.abs(x) < 1.0, 0.5 * tf.square(x), tf.abs(x) - 0.5)
    except:
        return tf.where(tf.abs(x) < 1.0, 0.5 * tf.square(x), tf.abs(x) - 0.5)

Für den Trainingsprozess muss zudem ein Optimizer definiert werden. Für dieses Netzwerk verwenden wir einen RMSProp-Optimierer. Allerdings verwenden wir eine statische Lernrate.

In [ ]:
# Define Optimizer
loss = None
target_value = None
chosen_action = None
optimizer = None
learning_rate_step = None
with tf.variable_scope("optimizer"):
    target_value = tf.placeholder("float32", [None], name="target_value")
    chosen_action = tf.placeholder("int64", [None], name="chosen_action")
    
    # Returns a tensor with the "action" index turned to 1 and all others set to 0
    action_one_hot = tf.one_hot(chosen_action, env.action_size, 1.0, 0.0, name="action_one_hot")
    q_acted = tf.reduce_sum(q * action_one_hot, reduction_indices=1, name="q_acted")
    
    delta = target_value - q_acted
    loss = tf.reduce_mean(error(delta), name="loss")
    
    # We are using an RMSProp-Optimizer with a variable learning rate.
    # The starting learning rate is reduced every n-th step by the decay rate to a set minimum.
    learning_rate = 0.00025
    learning_rate_minimum = 0.00025
    learning_rate_decay_step = 50000
    learning_rate_decay = 0.96
    
    learning_rate_step = tf.placeholder("int64", None, name="learning_rate_step")
    learning_rate_op = tf.maximum(learning_rate_minimum,
                                 tf.train.exponential_decay(
                                 learning_rate,
                                 learning_rate_step,
                                 learning_rate_decay_step,
                                 learning_rate_decay,
                                 staircase=True))
    optimizer = tf.train.RMSPropOptimizer(learning_rate_op, momentum=0.95, epsilon=0.01).minimize(loss)

Damit sind alle notwendigen Funktionen und Variablen des Graphen definiert.
Nun müssen sie nur noch initialisiert werden, bevor der Graph verwendet werden kann.

In [ ]:
# AUFGABE:
# Initialisiere alle Variablen mit TensorFlows globalen Variablen Initializer!

Zudem ist es sinnvoll die trainierten Gewichte zu speichern. Dafür wird ein Saver initialisert.

In [ ]:
saver = tf.train.Saver(W.values(), max_to_keep=10)

Nun kann der eigentliche Trainingsprozess gestartet werden:

In [ ]:
current_step = 0
learn_start = 5000
train_frequency = 4
target_network_rate = 1000
discount = 0.99
max_steps = 10000
save_rate = 5000
min_reward = -1.0
max_reward = 1.0

env.env.force_fps=True # enables a high framerate

history = History(config)
memory = ReplayMemory(config, ".")
state, reward, terminal = env.new_random_game() # start a new game in the environment

total_loss = 0.0
total_reward = 0.0
update_count = 0

for _ in range(config.history_length):
    history.add(state)

for i in range(0, max_steps):
    if current_step == learn_start:
        pass
    
    # AUFGABE:
    # Berechne das aktuelle Epsilon durch das die Chance bestimmt wird, eine zufällige Aktion auszuführen.
    # Zu Beginn des Trainings (current_step == 0), ist epsilon = 1.
    # Ab 1.000.000 Schritten gilt konstant epsilon = 0.1
    # Zwischen dem ersten und dem 1.000.000 Schritt soll epsilon linear abfallen.
    
    # predict an action
    if random.random() < epsilon:
        action = random.randrange(env.action_size)
    else:
        action = q_action.eval({prediction_input: [history.get()]})[0]
    
    # execute the action in the environment
    state, reward, is_terminal = env.act(action, is_training=True)
        
    # clip reward
    reward = max(min_reward, min(max_reward, reward))
    
    # observe outcome and save to memory
    history.add(state)
    memory.add(state, reward, action, is_terminal)
    
    if current_step > learn_start:
        if current_step % train_frequency == 0:
            # train prediction weights with minibatch
            prestates, actions, rewards, poststates, terminals = memory.sample()
            post_qs = target_q.eval({target_input: poststates})
            terminals = np.array(terminals) + 0.
            max_post_q = np.max(post_qs, axis=1)
            
            target_q_values = (1.0 - terminals) * discount * max_post_q + reward
            
            _, calculated_qs, calculated_loss = sess.run([optimizer, q, loss],
                                                        {target_value: target_q_values,
                                                        chosen_action: actions,
                                                        prediction_input: prestates,
                                                        learning_rate_step: current_step})
            total_loss += calculated_loss
            total_reward += reward
            update_count += 1
            
            avg_loss = total_loss / update_count
            
        if current_step % target_network_rate == target_network_rate - 1:
            # update the target network weights with the prediction network weights
            for name in W.keys():
                target_w_assign_op[name].eval({target_w_input[name]: W[name].eval()})
                
        if current_step % save_rate == 0:
            print "Saving Checkpoint %d. Average Reward: %.6f" % (current_step, (total_reward / save_rate))
            saver.save(sess, "./checkpoints/ckpt", global_step=current_step)
            total_reward = 0.0
    
    if is_terminal:
        # start new game
        state, reward, is_terminal = env.new_random_game()
    
    
    current_step += 1

In [ ]:
saver.restore(sess, "./checkpoints/ckpt-2000000")

Nach diesem Training wird der Agent getestet.

In [ ]:
history = History(config)
number_of_episodes = 2
epsilon = 0.1

env.env.force_fps=False # slow down the fps


for idx in xrange(number_of_episodes):
    state, reward, is_terminal = env.new_random_game()
    
    for _ in range(config.history_length):
        history.add(state)
        
    while(not is_terminal):
        # predict an action
        if random.random() < epsilon:
            action = random.randrange(env.action_size)
        else:
            action = q_action.eval({prediction_input: [history.get()]})[0]    
            
        # execute action
        state, reward, is_terminal = env.act(action, is_training=False)
        history.add(state)
        

Es kann hilfreich sein, den Trainingsverlauf mittels einer Summary und dem Tensorboard zu evaluieren.

**Aufgabe**: Fügen sie dem obigen Trainingsprozess eine ihrer Meinung sinnvolle Summary hinzu.